In [1]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta
import statsmodels.api as sm
import time 


In [2]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

In [3]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [4]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [5]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1
    
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [6]:
def doji(ohlc_df):
    """returns dataframe with doji candle column"""
    df = ohlc_df.copy()
    avg_candle_size = abs(df["close"] - df["open"]).median()
    df["doji"] = abs(df["close"] - df["open"]).iloc[-1] <= (0.05 * avg_candle_size)
    return df

def maru_bozu(ohlc_df):
    """returns dataframe with maru bozu candle column"""
    df = ohlc_df.copy()
    avg_candle_size = abs(df["close"] - df["open"]).median()
    df["h-c"] = df["high"] - df["close"]
    df["l-o"] = df["low"] - df["open"]
    df["h-o"] = df["high"] - df["open"]
    df["l-c"] = df["low"] - df["close"]
    df["maru_bozu"] = np.where(
        (df["close"] - df["open"] > 2 * avg_candle_size) &
        (df[["h-c", "l-o"]].max(axis=1) < 0.005 * avg_candle_size), "maru_bozu_green",
        np.where(
            (df["open"] - df["close"] > 2 * avg_candle_size) &
            (abs(df[["h-o", "l-c"]]).max(axis=1) < 0.005 * avg_candle_size), "maru_bozu_red", False
        )
    )
    df.drop(["h-c", "l-o", "h-o", "l-c"], axis=1, inplace=True)
    return df

def hammer(ohlc_df):
    """returns dataframe with hammer candle column"""
    df = ohlc_df.copy()
    df["hammer"] = (
        ((df["high"] - df["low"]) > 3 * (df["open"] - df["close"])) &
        ((df["close"] - df["low"]) / (.001 + df["high"] - df["low"]) > 0.6) &
        ((df["open"] - df["low"]) / (.001 + df["high"] - df["low"]) > 0.6) &
        (abs(df["close"] - df["open"]) > 0.1 * (df["high"] - df["low"]))
    ).iloc[-1]
    return df

def shooting_star(ohlc_df):
    """returns dataframe with shooting star candle column"""
    df = ohlc_df.copy()
    df["sstar"] = (
        ((df["high"] - df["low"]) > 3 * (df["open"] - df["close"])) &
        ((df["high"] - df["close"]) / (.001 + df["high"] - df["low"]) > 0.6) &
        ((df["high"] - df["open"]) / (.001 + df["high"] - df["low"]) > 0.6) &
        (abs(df["close"] - df["open"]) > 0.1 * (df["high"] - df["low"]))
    ).iloc[-1]
    return df

def levels(ohlc_day):
    """returns pivot point and support/resistance levels"""
    high = round(ohlc_day["high"].iloc[-1], 2)
    low = round(ohlc_day["low"].iloc[-1], 2)
    close = round(ohlc_day["close"].iloc[-1], 2)
    pivot = round((high + low + close) / 3, 2)
    r1 = round((2 * pivot - low), 2)
    r2 = round((pivot + (high - low)), 2)
    r3 = round((high + 2 * (pivot - low)), 2)
    s1 = round((2 * pivot - high), 2)
    s2 = round((pivot - (high - low)), 2)
    s3 = round((low - 2 * (high - pivot)), 2)
    return (pivot, r1, r2, r3, s1, s2, s3)

def trend(ohlc_df, n):
    """Function to assess the trend by analyzing each candle"""
    df = ohlc_df.copy()
    df["up"] = np.where(df["low"] >= df["low"].shift(1), 1, 0)
    df["dn"] = np.where(df["high"] <= df["high"].shift(1), 1, 0)
    if df["close"].iloc[-1] > df["open"].iloc[-1]:
        if df["up"].iloc[-n:].sum() >= 0.7 * n:
            return "uptrend"
    elif df["open"].iloc[-1] > df["close"].iloc[-1]:
        if df["dn"].iloc[-n:].sum() >= 0.7 * n:
            return "downtrend"
    else:
        return None
    
def res_sup(ohlc_df, ohlc_day):
    """calculates closest resistance and support levels for a given candle"""
    level = (
        ((ohlc_df["close"].iloc[-1] + ohlc_df["open"].iloc[-1]) / 2) +
        ((ohlc_df["high"].iloc[-1] + ohlc_df["low"].iloc[-1]) / 2)
    ) / 2
    p, r1, r2, r3, s1, s2, s3 = levels(ohlc_day)
    l_r1 = level - r1
    l_r2 = level - r2
    l_r3 = level - r3
    l_p = level - p
    l_s1 = level - s1
    l_s2 = level - s2
    l_s3 = level - s3
    lev_ser = pd.Series([l_p, l_r1, l_r2, l_r3, l_s1, l_s2, l_s3], index=["p", "r1", "r2", "r3", "s1", "s2", "s3"])
    sup = lev_ser[lev_ser > 0].idxmin()
    res = lev_ser[lev_ser < 0].idxmax()
    return (eval('{}'.format(res)), eval('{}'.format(sup)))

def candle_type(ohlc_df):
    """returns the candle type of the last candle of an OHLC DF"""
    candle = None
    if doji(ohlc_df)["doji"].iloc[-1] == True:
        candle = "doji"
    if maru_bozu(ohlc_df)["maru_bozu"].iloc[-1] == "maru_bozu_green":
        candle = "maru_bozu_green"
    if maru_bozu(ohlc_df)["maru_bozu"].iloc[-1] == "maru_bozu_red":
        candle = "maru_bozu_red"
    if shooting_star(ohlc_df)["sstar"].iloc[-1] == True:
        candle = "shooting_star"
    if hammer(ohlc_df)["hammer"].iloc[-1] == True:
        candle = "hammer"
    return candle

def candle_pattern(ohlc_df, ohlc_day):
    """returns the candle pattern identified"""
    pattern = None
    signi = "low"
    avg_candle_size = abs(ohlc_df["close"] - ohlc_df["open"]).median()
    sup, res = res_sup(ohlc_df, ohlc_day)

    # Check if the last close price is near the support or resistance levels
    if (sup - 1.5 * avg_candle_size) < ohlc_df["close"].iloc[-1] < (sup + 1.5 * avg_candle_size):
        signi = "HIGH"
    if (res - 1.5 * avg_candle_size) < ohlc_df["close"].iloc[-1] < (res + 1.5 * avg_candle_size):
        signi = "HIGH"

    # Determine the pattern based on the last candle type and other conditions
    if candle_type(ohlc_df) == 'doji' \
            and ohlc_df["close"].iloc[-1] > ohlc_df["close"].iloc[-2] \
            and ohlc_df["close"].iloc[-1] > ohlc_df["open"].iloc[-1]:
        pattern = "doji_bullish"

    if candle_type(ohlc_df) == 'doji' \
            and ohlc_df["close"].iloc[-1] < ohlc_df["close"].iloc[-2] \
            and ohlc_df["close"].iloc[-1] < ohlc_df["open"].iloc[-1]:
        pattern = "doji_bearish"

    if candle_type(ohlc_df) == "maru_bozu_green":
        pattern = "maru_bozu_bullish"

    if candle_type(ohlc_df) == "maru_bozu_red":
        pattern = "maru_bozu_bearish"

    if trend(ohlc_df.iloc[:-1, :], 7) == "uptrend" and candle_type(ohlc_df) == "hammer":
        pattern = "hanging_man_bearish"

    if trend(ohlc_df.iloc[:-1, :], 7) == "downtrend" and candle_type(ohlc_df) == "hammer":
        pattern = "hammer_bullish"

    if trend(ohlc_df.iloc[:-1, :], 7) == "uptrend" and candle_type(ohlc_df) == "shooting_star":
        pattern = "shooting_star_bearish"

    if trend(ohlc_df.iloc[:-1, :], 7) == "uptrend" \
            and candle_type(ohlc_df) == "doji" \
            and ohlc_df["high"].iloc[-1] < ohlc_df["close"].iloc[-2] \
            and ohlc_df["low"].iloc[-1] > ohlc_df["open"].iloc[-2]:
        pattern = "harami_cross_bearish"

    if trend(ohlc_df.iloc[:-1, :], 7) == "downtrend" \
            and candle_type(ohlc_df) == "doji" \
            and ohlc_df["high"].iloc[-1] < ohlc_df["open"].iloc[-2] \
            and ohlc_df["low"].iloc[-1] > ohlc_df["close"].iloc[-2]:
        pattern = "harami_cross_bullish"

    if trend(ohlc_df.iloc[:-1, :], 7) == "uptrend" \
            and candle_type(ohlc_df) != "doji" \
            and ohlc_df["open"].iloc[-1] > ohlc_df["high"].iloc[-2] \
            and ohlc_df["close"].iloc[-1] < ohlc_df["low"].iloc[-2]:
        pattern = "engulfing_bearish"

    if trend(ohlc_df.iloc[:-1, :], 7) == "downtrend" \
            and candle_type(ohlc_df) != "doji" \
            and ohlc_df["close"].iloc[-1] > ohlc_df["high"].iloc[-2] \
            and ohlc_df["open"].iloc[-1] < ohlc_df["low"].iloc[-2]:
        pattern = "engulfing_bullish"

    return "Significance - {}, Pattern - {}".format(signi, pattern)




In [7]:
tickers = ["ZEEL","WIPRO","VEDL","ULTRACEMCO","UPL","TITAN","TECHM","TATASTEEL",
           "TATAMOTORS","TCS","SUNPHARMA","SBIN","SHREECEM","RELIANCE","POWERGRID",
           "ONGC","NESTLEIND","NTPC","MARUTI","M&M","LT","KOTAKBANK","JSWSTEEL","INFY",
           "INDUSINDBK","IOC","ITC","ICICIBANK","HDFC","HINDUNILVR","HINDALCO",
           "HEROMOTOCO","HDFCBANK","HCLTECH","GRASIM","GAIL","EICHERMOT","DRREDDY",
           "COALINDIA","CIPLA","BRITANNIA","INFRATEL","BHARTIARTL","BPCL","BAJAJFINSV",
           "BAJFINANCE","BAJAJ-AUTO","AXISBANK","ASIANPAINT","ADANIPORTS","IDEA",
           "MCDOWELL-N","UBL","NIACL","SIEMENS","SRTRANSFIN","SBILIFE","PNB",
           "PGHH","PFC","PEL","PIDILITIND","PETRONET","PAGEIND","OFSS","NMDC","NHPC",
           "MOTHERSUMI","MARICO","LUPIN","L&TFH","INDIGO","IBULHSGFIN","ICICIPRULI",
           "ICICIGI","HINDZINC","HINDPETRO","HAVELLS","HDFCLIFE","HDFCAMC","GODREJCP",
           "GICRE","DIVISLAB","DABUR","DLF","CONCOR","COLPAL","CADILAHC","BOSCHLTD",
           "BIOCON","BERGEPAINT","BANKBARODA","BANDHANBNK","BAJAJHLDNG","DMART",
           "AUROPHARMA","ASHOKLEY","AMBUJACEM","ADANITRANS","ACC",
           "WHIRLPOOL","WABCOINDIA","VOLTAS","VINATIORGA","VBL","VARROC","VGUARD",
           "UNIONBANK","UCOBANK","TRENT","TORNTPOWER","TORNTPHARM","THERMAX","RAMCOCEM",
           "TATAPOWER","TATACONSUM","TVSMOTOR","TTKPRESTIG","SYNGENE","SYMPHONY",
           "SUPREMEIND","SUNDRMFAST","SUNDARMFIN","SUNTV","STRTECH","SAIL","SOLARINDS",
           "SHRIRAMCIT","SCHAEFFLER","SANOFI","SRF","SKFINDIA","SJVN","RELAXO",
           "RAJESHEXPO","RECLTD","RBLBANK","QUESS","PRESTIGE","POLYCAB","PHOENIXLTD",
           "PFIZER","PNBHOUSING","PIIND","OIL","OBEROIRLTY","NAM-INDIA","NATIONALUM",
           "NLCINDIA","NBCC","NATCOPHARM","MUTHOOTFIN","MPHASIS","MOTILALOFS","MINDTREE",
           "MFSL","MRPL","MANAPPURAM","MAHINDCIE","M&MFIN","MGL","MRF","LTI","LICHSGFIN",
           "LTTS","KANSAINER","KRBL","JUBILANT","JUBLFOOD","JINDALSTEL","JSWENERGY",
           "IPCALAB","NAUKRI","IGL","IOB","INDHOTEL","INDIANB","IBVENTURES","IDFCFIRSTB",
           "IDBI","ISEC","HUDCO","HONAUT","HAL","HEXAWARE","HATSUN","HEG","GSPL",
           "GUJGASLTD","GRAPHITE","GODREJPROP","GODREJIND","GODREJAGRO","GLENMARK",
           "GLAXO","GILLETTE","GMRINFRA","FRETAIL","FCONSUMER","FORTIS","FEDERALBNK",
           "EXIDEIND","ESCORTS","ERIS","ENGINERSIN","ENDURANCE","EMAMILTD","EDELWEISS",
           "EIHOTEL","LALPATHLAB","DALBHARAT","CUMMINSIND","CROMPTON","COROMANDEL","CUB",
           "CHOLAFIN","CHOLAHLDNG","CENTRALBK","CASTROLIND","CANBK","CRISIL","CESC",
           "BBTC","BLUEDART","BHEL","BHARATFORG","BEL","BAYERCROP","BATAINDIA",
           "BANKINDIA","BALKRISIND","ATUL","ASTRAL","APOLLOTYRE","APOLLOHOSP",
           "AMARAJABAT","ALKEM","APLLTD","AJANTPHARM","ABFRL","ABCAPITAL","ADANIPOWER",
           "ADANIGREEN","ADANIGAS","ABBOTINDIA","AAVAS","AARTIIND","AUBANK","AIAENG","3MINDIA"]


def main():
    for ticker in tickers:
        try:
            ohlc = fetchOHLC(ticker, '5minute',5)
            ohlc_day = fetchOHLC(ticker, 'day',30) 
            ohlc_day = ohlc_day.iloc[:-1,:]       
            cp = candle_pattern(ohlc,ohlc_day) 
            print(ticker, ": ",cp)   
        except Exception as e:
            print("skipping for ",ticker)
        
# Continuous execution        
starttime=time.time()
timeout = time.time() + 60*60*1  # 60 seconds times 60 meaning the script will run for 1 hr
while time.time() <= timeout:
    try:
        print("passthrough at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        main()
        time.sleep(300 - ((time.time() - starttime) % 300.0)) # 300 second interval between each new execution
    except KeyboardInterrupt:
        print('\n\nKeyboard exception received. Exiting.')
        exit() 

passthrough at  2024-06-10 19:18:42
ZEEL :  Significance - low, Pattern - None
WIPRO :  Significance - low, Pattern - None
skipping for  VEDL
ULTRACEMCO :  Significance - low, Pattern - None
UPL :  Significance - low, Pattern - None
TITAN :  Significance - low, Pattern - None
TECHM :  Significance - HIGH, Pattern - None
TATASTEEL :  Significance - low, Pattern - None
TATAMOTORS :  Significance - low, Pattern - None
TCS :  Significance - low, Pattern - None
SUNPHARMA :  Significance - low, Pattern - None
SBIN :  Significance - low, Pattern - None
skipping for  SHREECEM
RELIANCE :  Significance - low, Pattern - None
POWERGRID :  Significance - low, Pattern - None
ONGC :  Significance - low, Pattern - None
NESTLEIND :  Significance - HIGH, Pattern - None
NTPC :  Significance - low, Pattern - None
MARUTI :  Significance - HIGH, Pattern - None
M&M :  Significance - low, Pattern - None
LT :  Significance - low, Pattern - None
KOTAKBANK :  Significance - HIGH, Pattern - hammer_bullish
JSWSTEE